In [1]:
%load_ext autoreload
%autoreload 2

from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/')
import functions as fun

In [2]:
def load_csv(filename):
    file = open(filename, "r", encoding="utf-8")
    experiments = []
    headers = []
    reader = csv.reader(file)
    for i, lines in enumerate(reader):
        if i == 0:
            #headers = [e for e in lines[0].split(";")]
            h = lines[0].split(";")
            headers.append(h)
        else:
            tmp = lines[0].split(";")
            for j in range(0,len(tmp)):
                if (j != 0 and j != 1879):
                    tmp[j] = float(tmp[j])
                else:
                    continue
            #tmp = [float(tmp[j]) for j in range(0,len(tmp)) if (j!=0 and j!=1879)] #This removes two entries, dunno why. 
            experiments.append(tmp)
    file.close()
    return headers, experiments

def column_extraction(data_list,column_number): #columns start counting in 0
    col = []
    for i in range(0,len(experiments)):
        dato = experiments[i][column_number]
        col.append(dato)
    return col   

## Load both data sets for comparision

In [3]:
# Loading compound library
file_name = r'/home/jovyan/covid-data/specsSarsCov2-v1-CMPDannot.csv'# change it to the name of your excel file
specs = pd.read_csv(file_name)

In [4]:
#Loading Maris' data
headers, experiments = load_csv('/home/jovyan/covid-data/dalia_MeanFeaturesPerWell_BatchA.csv')

## Create the data sets to use 

### SPECS data set

In [5]:
# Loading data
print(f"There are {len(specs)} rows in SPECS")

There are 5283 rows in SPECS


In [6]:
# Remove outlier
print('Removing outlier...')
specs = specs.drop(specs.loc[specs['cbkid']=='CBK277930'].index.tolist())
print(f"There are {len(specs)} rows in SPECS")

# Remove duplicates but keeping the fisrts appereance 
print('Removing duplicates...')
specs = specs.drop_duplicates(subset = 'cbkid', keep = 'first')
print(f"There are {len(specs)} rows in SPECS")
specs.reset_index(drop=True, inplace=True)
specs.rename(columns = {'cbkid':'ID_library'}, inplace = True)

ids1 = specs["ID_library"]
if len(specs[ids1.isin(ids1[ids1.duplicated()])]) == 0:
    print("Now Covid data has zero repetitions")

Removing outlier...
There are 5281 rows in SPECS
Removing duplicates...
There are 5186 rows in SPECS
Now Covid data has zero repetitions


### Covid data set

In [7]:
# Loading dataframe
experiments_df = pd.DataFrame(experiments , columns=headers[0])
print(f"There are {len(experiments_df)} rows in Covid\n")

# Remove outlier
print('Removing outlier...')
covid_batchA_unique = experiments_df.copy().drop(experiments_df.loc[experiments_df['CompoundID']=='CBK277930'].index.tolist())
print(f"There are {len(covid_batchA_unique)} rows in Covid Batch A\n")

#Removing repetitions
covid_batchA_unique = covid_batchA_unique.drop_duplicates(subset=['CompoundID'], keep=False)
print(f"Removing repetitions...\nThere are {len(covid_batchA_unique)} rows in Covid Batch A\n")

ids_covid_unique = covid_batchA_unique["CompoundID"]
if len(covid_batchA_unique[ids_covid_unique.isin(ids_covid_unique[ids_covid_unique.duplicated()])]) == 0:
    print("Now Covid data has zero repetitions")

There are 6065 rows in Covid

Removing outlier...
There are 6063 rows in Covid Batch A

Removing repetitions...
There are 5089 rows in Covid Batch A

Now Covid data has zero repetitions


### Continuar con las repeticiones de covid

In [8]:
# Finding repetitions
covid_batchA_id_rep = experiments_df['CompoundID'].to_frame().groupby('CompoundID').size().sort_values(ascending=False).to_frame() #Count the elements in each group and sort
covid_batchA_id_rep.rename(columns = {0:'Count'}, inplace = True) #Renaming column
covid_batchA_id_rep.reset_index(inplace=True) # Ordering indexes
covid_batchA_id_rep

# Showing non-unique compounds
a = covid_batchA_id_rep[(covid_batchA_id_rep['Count'] >1) & (covid_batchA_id_rep['Count'] < 10)]
print(f'Printing repeated compounds\n {a.head(3)}\n')
repeated_compounds = a['CompoundID'].tolist() #Storing only repeated compounds
repeated_compounds.remove('CBK277930') # <------------------OUTLIER COMPOUND
print(f' There are {len(repeated_compounds)} repetitions in COVID')

Printing repeated compounds
   CompoundID  Count
7  CBK011558      3
8  CBK011603      3
9  CBK041250      3

 There are 87 repetitions in COVID


Now we have a file with no repetitions. The plan is to add one by one the median of the repeated compouns to it. 

In [9]:
covid_batchA = fun.select_median_v2(experiments = experiments_df, df_to_be_ready = covid_batchA_unique, repeated_compounds = repeated_compounds)
print(f"Adding medians...\nThere are {len(covid_batchA)} rows in Covid Batch A\n")

/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/functions.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_to_be_ready = df_to_be_ready.append(row)
/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/functions.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_to_be_ready = df_to_be_ready.append(row)
/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/functions.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_to_be_ready = df_to_be_ready.append(row)
/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/functions.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_to_be_ready = df_

Adding medians...
There are 5176 rows in Covid Batch A




Now, all the compounds in SPECS must also be in COVID to obtain the label when training. Let's proceed to compare and clean

In [10]:
#Adding labels to the classification problem. 1 is non-infected (MITO smaller that 0.0062) and 0 is infected. 
covid_batchA= fun.labelling(covid_batchA, len(covid_batchA.columns), label_positive = 1, label_negative = 0, threshold = 0.0062)#<-------------Change the threshold here
covid_batchA.rename(columns = {'CompoundID':'ID_covid'}, inplace = True)

In [11]:
#Comparing compopunds sss vs specs
comparision_specs_vs_sss = fun.library_to_experiment(covid_batchA,specs)
print(comparision_specs_vs_sss.head(2))

There are 14 compounds not founded in COVID
   Index in library       Name
0              1025  CBK041952
1              1257  CBK200552


In [12]:
name_list = comparision_specs_vs_sss['Name'].tolist()
print(f'SPECS has {len(specs)} rows')
for compound in name_list:
    specs = specs.drop(specs.loc[specs['ID_library'] == compound].index.tolist())
print(f'SPECS has {len(specs)} rows now')

SPECS has 5186 rows
SPECS has 5172 rows now


In [13]:
file_namn_specs = r'/home/jovyan/covid-data/SPECS_ready_POLINA.csv'
#specs.to_csv(file_namn_specs, sep=',', encoding='utf-8',index=False)
file_namn_sss = r'/home/jovyan/covid-data/COVID_BatchA_ready_POLINA.csv'
#covid_batchA.to_csv(file_namn_sss, sep=',', encoding='utf-8',index=False)

In [14]:
print('FINAL FILES READY')

FINAL FILES READY
